In [1]:
import torch 
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms
import torch.nn as nn
import os
from PIL import Image
from torch import optim
import torchvision

In [2]:
image_path="D:\RadarLab\LandmarksClassificationusingCNNs\landmark_images\landmark_images"
train_datapath=os.path.join(image_path,"train")
test_datapath=os.path.join(image_path,"test")
print(train_datapath)

D:\RadarLab\LandmarksClassificationusingCNNs\landmark_images\landmark_images\train


In [3]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:

classes=[]
trainimages_path=[]
testimages_path=[]

for path in os.listdir(train_datapath):
      classes.append(path)
      class_full_path= os.path.join(train_datapath,path)
      for i in os.listdir(class_full_path):
          full_img_path=os.path.join(class_full_path,i)
          trainimages_path.append(full_img_path)

for path in os.listdir(test_datapath):
    test_class_path = os.path.join(test_datapath, path)
    for img in os.listdir(test_class_path):
        testimages_path.append(os.path.join(test_class_path, img))

print(trainimages_path[-1])



D:\RadarLab\LandmarksClassificationusingCNNs\landmark_images\landmark_images\train\49.Temple_of_Olympian_Zeus\7f41eadaeb629930.jpg


In [5]:
idx_to_class = {i:j for i, j in enumerate(classes)}
class_to_idx = {value:key for key,value in idx_to_class.items()}

In [6]:
      
class Custom_Data(Dataset):
    
    def __init__(self,image_path,transforms):
        super().__init__()
        self.image_path=image_path
        self.transforms=transforms
    def __len__(self):
        return len(self.image_path)
    def __getitem__(self,index):
        image_path=self.image_path[index]
        image = Image.open(image_path)
        #image.show()
        label = image_path.split('\\')[-2]
        label = class_to_idx[label]
        if self.transforms is not None:
            image = self.transforms(image)

        return image,label

In [9]:
tf=transforms.Compose([   
         #transforms.ToPILImage(),
         transforms.Resize(size=(224, 224)),
         transforms.ToTensor(),
         transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
    ])



train_dataset=Custom_Data(trainimages_path,tf)
test_dataset=Custom_Data(trainimages_path,tf)
print(len(train_loader))

50


In [8]:
train_loader= DataLoader(train_dataset,batch_size=100,shuffle=True)
test_loader= DataLoader(test_dataset,batch_size=100,shuffle=False)

In [10]:
batch = next(iter(train_loader))
images, label = batch
images.shape
label.shape


torch.Size([100])

In [75]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=3,  # 1 for grey scale, 3 for RGB  , #n for n number of features maps
                out_channels=16,    # number of feature maps
                kernel_size=3,         # kernel size
                stride=1,
                padding=1, # (K-1)/2
                #output size=   224(inputsise) - 3(kernel) + 2*0(padding) / 1(stride) +1  result 8 feature maps of 224x224
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),    # 224x224 reduced to 112x112
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32 ,3, 1, 1),    # 32 feature maps of 112x112
            nn.ReLU(),
            nn.MaxPool2d(2),  # (generated feature map of size= (14-5+4) /2 )  After pooling = 56x56
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 64 ,3 , 1, 1),    # 64 feature maps with size 56x56 produced by filter 
            nn.ReLU(),
            nn.MaxPool2d(2), 
        )
        # fully connected layer, output 10 classes
        self.out1 = nn.Linear(64*28*28 , 224)
        self.dropout= nn.Dropout(0.3)
        self.out2= nn.Linear(224 , 50)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x) # return (batch,channel,height,width)
        x = torch.flatten(x,1)
        output = self.out1(x)
        output = self.dropout(output)
        output = self.out2(output)
        output = nn.Softmax(output)
        
        return output  # return x for visualization

In [76]:
model = CNN().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.01)
num_epochs = 10
batch_size=100

In [77]:
dummy_input = torch.randn(1, 3, 224, 224).to(device) 
 # Adjust size as needed
model(dummy_input)

Softmax(
  dim=tensor([[-0.1022, -0.0784,  0.0134,  0.0643,  0.0960, -0.0945,  0.1736, -0.1053,
           -0.0392, -0.1068, -0.0907, -0.0425,  0.0420,  0.0866,  0.0286,  0.0380,
            0.0568, -0.0970,  0.0514,  0.0154, -0.0106,  0.1359,  0.0063, -0.0311,
            0.0185,  0.0139, -0.0648, -0.0130, -0.1428,  0.0777, -0.0032,  0.0641,
           -0.0511, -0.0997,  0.0753,  0.0554,  0.0322, -0.0335,  0.0042, -0.1518,
            0.1379, -0.0762,  0.0751, -0.0433,  0.0580, -0.0333, -0.1061, -0.0585,
           -0.0855, -0.0083]], grad_fn=<AddmmBackward0>)
)

In [53]:
def train(dataloader,model,loss_fn,optimizer):
    size=len(dataloader.dataset)

    model.train()

    for batch,(X,y) in enumerate(dataloader):
        X=X.to(device)
        y=y.to(device)
        #forward pass
        y_pred=model(X)
        #loss
        loss=loss_fn(y_pred,y)
        #backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


        if batch % 10 == 0:
                loss, current = loss.item(), batch * batch_size + len(X)
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [54]:

for z in range(num_epochs):
    print(f"Epoch {z+1}\n-------------------------------")
    train(train_loader, model, loss_fn, optimizer)

print("Done!")


Epoch 1
-------------------------------
loss: 3.913688  [  100/ 4996]
loss: 3.911410  [ 1100/ 4996]
loss: 3.920201  [ 2100/ 4996]
loss: 3.914086  [ 3100/ 4996]
loss: 3.899322  [ 4100/ 4996]
Epoch 2
-------------------------------
loss: 3.847996  [  100/ 4996]
loss: 3.839329  [ 1100/ 4996]
loss: 3.816904  [ 2100/ 4996]
loss: 3.803485  [ 3100/ 4996]
loss: 3.829708  [ 4100/ 4996]
Epoch 3
-------------------------------
loss: 3.722653  [  100/ 4996]
loss: 3.636260  [ 1100/ 4996]
loss: 3.615500  [ 2100/ 4996]
loss: 3.571392  [ 3100/ 4996]
loss: 3.598412  [ 4100/ 4996]
Epoch 4
-------------------------------
loss: 3.455754  [  100/ 4996]
loss: 3.285309  [ 1100/ 4996]
loss: 3.099061  [ 2100/ 4996]
loss: 3.481343  [ 3100/ 4996]
loss: 3.396529  [ 4100/ 4996]
Epoch 5
-------------------------------
loss: 2.988665  [  100/ 4996]
loss: 2.645732  [ 1100/ 4996]
loss: 3.017979  [ 2100/ 4996]
loss: 2.866925  [ 3100/ 4996]
loss: 2.970355  [ 4100/ 4996]
Epoch 6
-------------------------------
loss: 2.72

In [55]:
def test():
    # Test the model
    best_loss=float("inf")
    test_loss=0
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            test_output = model(images)
            test_loss +=loss_fn(test_output,labels).item()
            pred_y = torch.max(test_output, 1)[1].data.squeeze()

            if test_loss<best_loss:
                best_loss=test_loss
                torch.save(model.state_dict(),"model_weight.pth")

            accuracy = (pred_y == labels).sum().item() / float(labels.size(0))
        print(accuracy) 


test()
print()

0.78125



In [57]:
model.load_state_dict(torch.load('model_weight.pth'))
torch.save(model,'model.pth')
model.eval()

CNN(
  (conv1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out1): Linear(in_features=50176, out_features=224, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (out2): Linear(in_features=224, out_features=50, bias=True)
)

In [59]:
transform = transforms.Compose([# Ensure the image is in grayscale if needed
    transforms.Resize((224, 224)),  # Resize to the same size as the training images
    transforms.ToTensor(), # Convert to tensor
])


In [72]:
# Load and preprocess the test image
image = Image.open('test_image.jpeg')
image = transform(image).unsqueeze(0)

In [73]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
image = image.to(device)

In [74]:
# Classify the image
with torch.no_grad():
    output = model(image)
    predicted_class = torch.argmax(output, dim=1)

print(f'Predicted class: {predicted_class.item()}')

Predicted class: 3
